In [1]:
## General librairies
import pandas as pd
import numpy as np
import _pickle as pickle
import matplotlib.pyplot as plt
import random
import sys
import time
from copy import deepcopy
import os

from sklearn.metrics import cohen_kappa_score

from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import json

import dateutil.parser

Using TensorFlow backend.


In [2]:
specs = 'specs.csv'
test = 'test.csv'
train = 'train.csv'
train_labels = 'train_labels.csv'
subs = 'sample_submission_exemple.csv'



In [3]:
def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

def relative_time(x):
    x1 = []
    for elt in x:
        x1.append((elt-x[0]).item()/1000000000)
    return x1

def categorise(x):
    dico = {}
    count = 0
    for elt in x:
        if not(elt in dico):
            dico[elt] = count
            count += 1
    return dico

def padding( dataset, n):
    d = list(np.zeros(len(dataset)))
    c = 0
    count = 0
    for elt in dataset:
        elt = elt.values
        if count % 100 == 0:
            print(count)
        u = elt.shape[0]
        
        if u<200:
            c += 1
        
        if u > n:
            d[count] = elt[-n:]
        else:
            a = np.zeros(((n-u), elt.shape[1])) -1
            elt = np.concatenate([a, elt], axis = 0)
            d[count] = elt.tolist()
        count += 1
    return d

In [ ]:
df = pd.read_csv(train)

dtitle = categorise(df['title'])
dtype = categorise(df['type'])
dworld = categorise(df['world'])
devent = categorise(df['event_code'])
devent_id = categorise(df['event_id'])

vect = df['event_code']
vect[df['event_id'] == '17113b36'] = 4100
df['event_code'] = vect

# df['date'] = df['timestamp'].apply(dateutil.parser.parse)


save((dtitle, dtype, dworld, devent, devent_id), 'dicos cats')

In [ ]:
df.columns

In [ ]:
assessments = df[df['type']=='Assessment']['title'].unique()
games = df[df['type']=='Games']['title'].unique()

def build_line(df):
    vect = []
    
    ## Categorical
    vect.append(df.iloc[0]['installation_id'])
    vect.append(df.iloc[0]['game_session'])
    vect.append(df.iloc[0]['title'])
    vect.append(df.iloc[0]['type'])
    vect.append(df.iloc[0]['world'])
    vect.append(df.iloc[0]['timestamp'])
    
    ## Assessment_accuracy_group
    
    if df.iloc[0]['title'] in assessments:
        df1 = df[df['event_code'] == 4100]
        if df1.shape[0] == 0:
            vect.append(0)
            vect.append(0)
        else:
            n_corr = 0
            n_inc = 0
            for i in df1['event_data']:
                a = json.loads(i)['correct']
                if a:
                    n_corr += 1
                else:
                    n_inc += 1
            acc = n_corr/(n_corr+n_inc)    
            if acc == 1:
                acc_class = 3
            elif acc == 0.5:
                acc_class = 2
            elif acc <=0:
                acc_class = 0
            else:
                acc_class = 1
            vect.append(acc_class)
            vect.append(acc)
    else:
        vect.append(-1)
        vect.append(-1)
    
    ## feedbacks_accuracy
    if df.iloc[0]['title'] in assessments or df.iloc[0]['title'] in games:
        vc = df['event_code'].value_counts()
        if 3021 in vc.index:
            n_corr = vc[3021]
        else:
            n_corr = 0
        
        if 3020 in vc.index:
            n_inc = vc[3020]
        else:
            n_inc = 0
        
        if n_corr + n_inc == 0:
            vect.append(0)
        else:
            vect.append(n_corr/(n_corr+n_inc))
    else:
        vect.append(0)
    ## Time
    vect.append(df['game_time'].max()-df['game_time'].min())
    
    ## Event_code
    vc = df['event_code'].value_counts()
    
    for elt in devent:
        if elt in vc.index:
            vect.append(vc[elt])
        else:
            vect.append(0)
    
    ## Event_ID
    vc = df['event_id'].value_counts()
    for elt in devent_id:
        if elt in vc.index:
            vect.append(vc[elt])
        else:
            vect.append(0) 
            
    return vect

columns = ['installation_id', 'game_session', 'title', 'type', 'world', 'timestamp', 'accuracy_group', 'accuracy', 'feed_acc','time']

for elt in devent:
    columns.append('n_'+str(elt))
for elt in devent_id:
    columns.append('n_'+str(elt))

In [ ]:
dataset = list(np.zeros(df['game_session'].unique().shape[0]))

count = 0
for i, session in df.groupby(['game_session']):
#     if count >= 100:
#         break
    line =build_line(session)
#     dataset.append(line)
    dataset[count] = line
    if count % 1000 == 0:
        print(count)
    count += 1

dataset = pd.DataFrame(dataset, columns = columns)
dataset.to_csv('data by session events')

In [ ]:
save(dataset, 'data by session events')

In [ ]:
df = load('data by session events')
labels = pd.read_csv(train_labels)

ids = df[df['type']=='Assessment']['installation_id'].unique()
df = df[df.installation_id.isin(ids)]
df = df[df.installation_id.isin(labels.installation_id.unique())]

df['date'] = df['timestamp'].apply(dateutil.parser.parse)

dtitle, dtype, dworld, devent, devent_id = load('dicos cats')

df = df.replace({'title' : dtitle})
df = df.replace({'type' : dtype})
df = df.replace({'world' : dworld})

In [ ]:
dataset = list(np.zeros(labels.shape[0]))

for i in range(labels.shape[0]):
    
    if i%100 ==0:
        print(i)
    installation_id = labels.iloc[i]['installation_id']
    game_session = labels.iloc[i]['game_session']
    game_title = dtitle[labels.iloc[i]['title']]
    
    df1 = df[df['installation_id'] == installation_id]
    
    time = df1[df1['game_session'] == game_session].iloc[0]['date']
    
    df1 = df1[df1['date'] < time]
    
    df1 = df1.sort_values(by = ['date'])
    
    def sub(x, time = time):
        return (time - x).total_seconds()/3600
    
    df1['date'] = df1['date'].apply(sub)
    
    df1['pred_title'] = game_title
    
    drop = ['installation_id', 'game_session', 'pred_title', 'timestamp']
    cols = ['pred_title'] + [elt for elt in columns if not(elt in drop)] + ['date']
    
    df1 = df1[cols]
    
    dataset[i] = df1

In [ ]:
save(dataset, 'train set non padded')


In [ ]:
dataset[0].columns

In [4]:
def augment(X, y, N):
    y1 = []
    X1 = list(np.zeros(N))
    
    
    
    for i in range(N):
        if i%1000 == 0:
            print(i)
        ind = random.randint(0, len(X)-1)
        df = X[ind]
        
        df = df.reset_index(drop = True)
        indices = []
        for j in df.index:
            a = random.uniform(0,1)
            if a < 0.2:
#                 print(i)
                indices.append(j)
        df = df.drop(indices)
#         df = df.sample(frac = 1)
        
        X1[i] = df
        y1.append(y[ind])
    return X1, y1
    
def format_input(X, pad):
    i = X.shape[0]
    j = X.shape[1]
    return [X[:, :, 0].reshape((i,j,1)), X[:,:, 1].reshape((i,j,1)),X[:,:, 2].reshape((i,j,1)),X[:,:, 3].reshape((i,j,1)),X[:,:, 4].reshape((i,j,1)),X[:,:, 5:]]

In [5]:
keep = ['pred_title',
 'title',
 'type',
 'world',
 'accuracy_group',
 'accuracy',
 'feed_acc',
 'time',
 'n_2000',
 'n_3010',
 'n_3110',
 'n_4070',
 'n_4090',
 'n_4030',
 'n_4035',
 'n_4021',
 'n_4020',
 'n_4010',
 'n_2080',
 'n_2083',
 'n_2040',
 'n_2020',
 'n_2030',
 'n_3021',
 'n_3121',
 'n_2050',
 'n_3020',
 'n_3120',
 'n_2060',
 'n_2070',
 'n_4031',
 'n_4025',
 'n_5000',
 'n_5010',
 'n_2081',
 'n_2025',
 'n_4022',
 'n_2035',
 'n_4040',
 'n_4100',
 'n_2010',
 'n_4110',
 'n_4045',
 'n_4095',
 'n_4220',
 'n_2075',
 'n_4230',
 'n_4235',
 'n_4080',
 'n_4050',
 'date']

In [6]:
dataset = load('train set non padded light')

# dataset1 = list(np.zeros(len(dataset)))
# for i in range(len(dataset)):
#     df = dataset[i]
#     df = df[keep]
#     dataset1[i] = df

# dataset = dataset1

# save(dataset, 'train set non padded light')

labels = pd.read_csv(train_labels)

Y = labels['accuracy_group'].values.astype('float64')

y = np_utils.to_categorical(Y)

X_train, X_test, y_train, y_test = train_test_split(dataset, Y, test_size=0.2, random_state=43)
# del(dataset)

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y),
                                               Y)

In [7]:
# X_train, y_train = augment(X_train, y_train, 100000)

pad = 100
X_train = padding(X_train, pad)
X_test = padding(X_test, pad)

X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = format_input(X_train, pad)
X_test = format_input(X_test, pad)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500


In [8]:
from keras.models import Model, Sequential
from keras.layers import Input, Dropout, Dense,CuDNNLSTM, Flatten, Embedding, TimeDistributed, Concatenate, LSTM, BatchNormalization, Lambda, Reshape
from keras.regularizers import l2

pad_size = pad

num = Input(shape = (pad_size, 46))
pred_title = Input(shape = (pad_size, 1))
title = Input(shape = (pad_size, 1))
type1 = Input(shape = (pad_size, 1))
world = Input(shape = (pad_size, 1))
acc_grp = Input(shape = (pad_size, 1))

inputs = [pred_title, title, type1, world,acc_grp, num]

num_emb = TimeDistributed(Dense(16))(num)

pred_title_emb = TimeDistributed(Embedding( 5 ,5))(pred_title)
pred_title_emb = Lambda(lambda x: x, output_shape = lambda s:s)(pred_title_emb)
pred_title_emb = Reshape((-1,5))(pred_title_emb)

title_emb = TimeDistributed(Embedding( 44 ,10))(title)
title_emb = Lambda(lambda x: x, output_shape = lambda s:s)(title_emb)
title_emb = Reshape((-1,10))(title_emb)

type_emb = TimeDistributed(Embedding( 4 ,4))(type1)
type_emb = Lambda(lambda x: x, output_shape = lambda s:s)(type_emb)
type_emb = Reshape((-1,4))(type_emb)

world_emb = TimeDistributed(Embedding( 4 ,2))(world)
world_emb = Lambda(lambda x: x, output_shape = lambda s:s)(world_emb)
world_emb = Reshape((-1,2))(world_emb)

acc_grp_emb = TimeDistributed(Embedding( 5 ,3))(acc_grp)
acc_grp_emb = Lambda(lambda x: x, output_shape = lambda s:s)(acc_grp_emb)
acc_grp_emb = Reshape((-1,3))(acc_grp_emb)

embedded = [pred_title_emb, title_emb, type_emb, world_emb, acc_grp_emb]

merged = Concatenate()(embedded + [num_emb])

merged = BatchNormalization()(merged)

merged = TimeDistributed(Dense(16))(merged)
merged = TimeDistributed(Dense(5))(merged)

lstm_out = CuDNNLSTM(64, return_sequences = True)(merged)
lstm_out = CuDNNLSTM(64, return_sequences = True)(lstm_out)
lstm_out = CuDNNLSTM(64, return_sequences = False)(lstm_out)

# lstm_out = Flatten()(lstm_out)

td = Dropout(0.2)(lstm_out)
td = Dense(32, activation = 'relu')(td)
td = Dropout(0.2)(td)

td = Dense(1, activation = 'linear')(td)

model = Model(inputs , td)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_6 (I

In [ ]:
import keras
from sklearn.metrics import cohen_kappa_score
import keras.backend as K
from keras.optimizers import SGD

optimizer = SGD(0.01)
model.compile(loss='mse', #'categorical_crossentropy',
              optimizer=optimizer,
              metrics=['mse'])

batch_size = 64
epochs = 50

history = model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test))#, class_weight=class_weights)



Train on 14152 samples, validate on 3538 samples
Epoch 1/50
14152/14152 [==============================] - 9s 650us/step - loss: 1.7461 - mean_squared_error: 1.7461 - val_loss: 1.5394 - val_mean_squared_error: 1.5394
Epoch 2/50
14152/14152 [==============================] - 5s 378us/step - loss: 1.5622 - mean_squared_error: 1.5622 - val_loss: 1.4877 - val_mean_squared_error: 1.4877
Epoch 3/50
14152/14152 [==============================] - 5s 378us/step - loss: 1.4924 - mean_squared_error: 1.4924 - val_loss: 1.4044 - val_mean_squared_error: 1.4044
Epoch 4/50
14152/14152 [==============================] - 5s 383us/step - loss: 1.4474 - mean_squared_error: 1.4474 - val_loss: 1.3604 - val_mean_squared_error: 1.3604
Epoch 5/50
14152/14152 [==============================] - 5s 378us/step - loss: 1.4030 - mean_squared_error: 1.4030 - val_loss: 1.3927 - val_mean_squared_error: 1.3927
Epoch 6/50
14152/14152 [==============================] - 5s 380us/step - loss: 1.3738 - mean_squared_error: 

In [ ]:
pred = model.predict(X_test).reshape(len(X_test[0]))

In [ ]:
pred

In [ ]:
y_test

In [ ]:
import optuna
def tres(pred, t):
    pred1 = deepcopy(pred)
    pred1[pred1 < t[0]] = 0
    pred1[(pred1>= t[0])&(pred1 < t[1])] = 1
    pred1[(pred1>= t[1])&(pred1 < t[2])] = 2
    pred1[pred1 >= t[2]] = 3
    
    return pred1

p = model.predict(X_train).reshape(len(X_train[0]))

def objective(trial, pred =p,  y_test = y_train):
    x0 = trial.suggest_uniform('x0', pred.min(), pred.max())
    x1 = trial.suggest_uniform('x1', x0,  pred.max())
    x2 = trial.suggest_uniform('x2', x1,  pred.max())

    t = [x0, x1, x2]
    pred1 = tres(pred, t)
    a = cohen_kappa_score(y_test, pred1, weights = 'quadratic')
    print(a)
    return 1-a

study = optuna.create_study()
study.optimize(objective, n_trials=300)

In [ ]:
t = list(study.best_params.values())

pred = model.predict(X_test).reshape(len(X_test[0]))
pred1 = tres(pred, t)

print(cohen_kappa_score(y_test, pred1, weights = 'quadratic'))